<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/topic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dev Environment

In [1]:
!pip install tomotopy
!pip install hangul_utils

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [4]:
import nltk
from nltk.corpus import stopwords
from pykospacing import Spacing
from khaiii import KhaiiiApi
from hangul_utils import split_syllables, join_jamos

In [5]:
import numpy as np
import pandas as pd
import tomotopy as tp

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 2. Import Data

In [8]:
tweets = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/election_sample/tweet.csv')

In [9]:
tweets['text']

0       윤석열이랑 이재명이랑 사랑의 도피해서 심상정이 대통령된 다음에 동성혼 법제화 시켜서...
1       [속보] 20대 대통령 선거 후보 확정\n더불어민주당 이재명\n국민의힘 윤석열\n정...
2       정확히 표현해야 합니다.\n\n- 이재명을 안찍는건 당연.\n- 기권을 하면 이재명...
3       이 심상정의 1분을 어떻게 잊을 수 있을까. 나는 절대 못 잊는다. https://...
4       심상정을 뽑는 표가 사표가 아니란걸 우리 여성들이 보여줘야 한다. 인구 절반의 여성...
                              ...                        
5316    윤석열은 끊임없이 까면서\n이재명에 대해 일언반구도 없는\n당신도 똑같은 역사의 죄...
5317    💚다음주 토요일 서초집회 아젠다💚\n📌대장동 특검\n📌합수부 설치\n📌이재명 구속\...
5318    아니 애초에 이재명이 민주당안밖에서 반문질로 지금까지 커온거 사실인데 무슨 문재인을...
5319    여기는 경북울진 주변에 많은 사람들이 공통적으로 하는말 “윤석렬 너무 수준떨어진다”...
5320    김종혁 전 국장 "정진상이 뭔가 고리가 돼 있거나 의심받고 있는데 떡 하니 부실장으...
Name: text, Length: 5321, dtype: object

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
stopwords = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/stopwords.csv')

# 3. Data Preprocessing

In [22]:
spacing = Spacing()
api = KhaiiiApi()

In [23]:
relations = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']           # 관계언 (조사)
dependents = ['EP', 'EF', 'EC', 'ETN', 'ETM', 'XPN', 'XSN', 'XSV', 'XSA', 'XR']     # 의존격 (어미, 접사)
symbols = ['SF', 'SP', 'SS', 'SE', 'SO', 'SL', 'SH', 'SW', 'SWK', 'SN']             # 기호

In [28]:
unused = []
unused.extend(relations)
unused.extend(dependents)
unused.extend(symbols)

In [24]:
def flatten(list_of_lists):
    '''
    Return flattened list from list of lists.
    '''
    return [y for x in list_of_lists for y in x]

In [25]:
def khaiii_analyze(phrase):
    '''
    Return Lexicons analyzed by Khaiii Analyzer.
    '''
    return [z.lex for z in flatten([y.morphs for y in api.analyze(phrase)]) if z.tag not in unused]

In [26]:
tweets['original'] = tweets['text']

In [27]:
# Erase Non-Korean and Non-English Keywords
tweets['text'] = tweets['text'].str.replace('[^A-Za-z가-힣0-9ㄱ-ㅎㅏ-ㅣ.,/ ]', ' ')

In [33]:
tweets['text'] = tweets['text'].map(split_syllables)

In [34]:
tweets['text'] = tweets['text'].map(join_jamos)

In [36]:
tweets['text'] = tweets['text'].map(lambda x : x.replace(' ', ''))

In [38]:
tweets['text'] = tweets['text'].map(spacing)

In [39]:
tweets['text'] = tweets['text'].map(lambda x : ' '.join(khaiii_analyze(x) if x != '' else ''))

In [40]:
tweets['text']

0       윤석열 이재명 사랑 도 피하 심상정 대통령 되 다음 동성혼법 제화 서 둘 결혼식 부...
1       속보 대 대통령 선거 후보 확정 더불 민주 당이재 명 국민 힘 윤석 열정 당 심상정...
2       정확히 표현 하 이재명 안 찍 것 당연 기권 하 이재명 도와주 것 투표 불참 또한 ...
3                               이 심상정 분 어떻 잊 수 있 나 절대 못 잊
4       심상정 뽑 표 사표 아니 것 우리 여성 보이 주 하 인구 절반 여성 사표 이 협박 ...
                              ...                        
5316              윤석열 끊임없이 까 이재명 대하 일언반구 없 당신 똑같 역사 죄인 이이
5317    다음 주 토요일 서초집회 아젠다대 장 동 특검 합수부 설치 이재명 구속 송영길 탄핵...
5318    아니 애초 이재명 민주당 안 반문질 지금 커 오 거 사실 이 무슨 문재인 생각 이재...
5319    여기 경북 울진 주변 많 사람 공통 하 말 윤석렬 너무 수준 떨어지 다 반응 내 그...
5320    김종혁 전 국장 정진상 뭐 이 고리 되 있 의심 받 있 떡 하 부실장 앉히 것 뭐 ...
Name: text, Length: 5321, dtype: object

In [ ]:
print('#4: Tokenization ->', df['text'][0])
df['text'] = df['text'].apply(lambda x : [y for y in x.split(' ') if y not in stopwords])
print('#5: Stopword ->', df['text'][0])

# %% In[4]: Train Model

model = tp.LDAModel(k = 20, alpha = 0.5)

for i in range(len(df)):
    if df['text'][i] != []:
        model.add_doc(df['text'][i])

for i in range(0, 100, 10):
    model.train(10)
    print(f'Iteration: {i}, Log-Likelihood: {model.ll_per_word}')

# %% In[5]: Show Train Results
# for k in range(model.k):
#     print(f'Top 10 words of topic #{k}')
#     print(model.get_topic_words(k, top_n = 10))

# %% In[6]: Save Temp Model
model.save('./AI/topic_model.bin')
